# Run Pixel Classifier on a Test Sites
Note: The script `nn_predict.py` also works well for this purpose if evaluating a single site.

This functionality is also contained within the `train_pixel_classifier` notebook for the purposes of evaluating a trained model.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import pickle

import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras

from scripts.dl_utils import download_patch, rect_from_point
from scripts.nn_predict import make_predictions, visualize_predictions

In [ ]:
rect_width = 0.02
coords = [116.0908,-8.6451]
start_date = '2020-05-01'
end_date = '2020-06-01'
patches = download_patch(rect_from_point(coords, rect_width), start_date, end_date)

In [ ]:
model = keras.models.load_model('../models/model_v1.1.8_5-07-21.h5')
pred_stack = make_predictions(patches, model)
visualize_predictions(patches, pred_stack, threshold=0.8)

In [ ]:
# Compare to a baseline model
baseline_model = keras.models.load_model('../models/65_mo_tpa_bootstrap_toa-12-20-2020.h5')
pred_stack_baseline = make_predictions(patches, baseline_model)
visualize_predictions(patches, pred_stack_baseline, threshold=0.8)

### Show timeseries predictions

In [ ]:
threshold = 0.8
num_img = int(np.ceil(np.sqrt(len(patches))))

plt.figure(figsize=(num_img,num_img), dpi=250, facecolor=(1,1,1))
for i, (img, pred) in enumerate(zip(patches, pred_stack)):
    rgb = img[:,:,3:0:-1] / 3000
    rgb[pred > threshold, 0] = 0.9
    rgb[pred > threshold, 1] = 0
    rgb[pred > threshold, 2] = 0.1
    
    plt.subplot(num_img, num_img, i + 1)
    plt.imshow(np.clip(rgb, 0, 1))
    plt.axis('off')
plt.show()